<a href="https://colab.research.google.com/github/Kim-Jeong-Ju/AI_Modeling_NLP/blob/main/Count_based_Word_Expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Packages and Modules Importation**

In [ ]:
import numpy as np
import pandas as pd
from math import log

import nltk
nltk.download('stopwords')

!pip install konlpy

from nltk.corpus import stopwords
from konlpy.tag import Okt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 986 kB/s 
     |████████████████████████████████| 453 kB 61.8 MB/s 


# **BoW, Bag of Words**  


---

- BoW는 **각 단어가 등장한 횟수를 수치화**하는 텍스트 표현 방법, 주로 어떤 단어가 얼마나 등장했는지를 기준으로 **문서가 어떤 성격의 문서인지를 판단**하는 작업에 쓰임  
- **분류** 문제 or **여러 문서 간의 유사도**를 구하는 문제에 주로 쓰임  
> *cf) BoW 사용 예시*  
> *가령, '달리기', '체력', '근력'과 같은 단어가 자주 등장하면 해당 문서를 체육 관련 문서로 분류할 수 있음. '미분', '방정식', '부등식'과 같은 단어가 자주 등장한다면 수학 관련 문서로 분류할 수 있음*.

In [ ]:
okt = Okt()

def build_bow(document):
  document = document.replace(".", "")      # 온점(.) 제거
  document_tkn = okt.morphs(document)       # 한국어 형태소 분석 -> 토큰화

  word_to_index = {}
  bow = []

  for word in document_tkn:
    if word not in word_to_index.keys():            # 한번도 등장하지 않았던 단어이면
      word_to_index[word] = len(word_to_index)      # vocab에 추가
      bow.append(1)                                 # BoW에 추가하여 빈도수 1로 초기화
    else:                                           # 이미 등장했던 단어이면
      index = word_to_index.get(word)               # vocab 내 해당 단어의 word에 해당하는 key값 반환
      bow[index] += 1                               # Bow에 빈도수 +1 추가
  
  return word_to_index, bow

In [ ]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab1, bow1 = build_bow(doc1)

print('Vocabulary =', vocab1)
print('BoW vector =', bow1)

Vocabulary = {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
BoW vector = [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [ ]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
vocab2, bow2 = build_bow(doc2)

print('Vocabulary =', vocab2)
print('BoW vector =', bow2)

Vocabulary = {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
BoW vector = [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [ ]:
doc3 = doc1 + ' ' + doc2
vocab3, bow3 = build_bow(doc3)

print('Vocabulary =', vocab3)
print('BoW vector =', bow3)

Vocabulary = {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
BoW vector = [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


## **CountVectorizer Class를 이용한 BoW 생성**  
Scikit-learn Package에서는 단어의 빈도를 Count하여 Vector화하는 **CountVectorizer Class**를 지원함  
- 알파벳 I는 BoW를 만드는 과정에서 사라졌는데, 이는 CountVectorizer가 기본적으로 **길이가 2이상인 문자에 대해서만 토큰으로 인식**하기 때문  
- CountVectorizer는 **단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화**를 진행하고 BoW를 만듦  
- 한국어에 CountVectorizer를 적용하면, 조사 등의 이유로 제대로 BoW가 만들어지지 않음

In [ ]:
corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

print('BoW vector =', vector.fit_transform(corpus).toarray())      # 코퍼스로부터 각 단어의 빈도수를 기록

print('Vocabulary =',vector.vocabulary_)                                    # 각 단어의 인덱스가 어떻게 부여되었는지를 출력

BoW vector = [[1 1 2 1 2 1]]
Vocabulary = {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


## **Stop Words를 제거한 BoW 생성**

In [ ]:
text = ["Family is not an important thing. It's everything."]

vector1 = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])     # 직접 Stop Words를 지정하는 방법
print('BoW vector =',vector1.fit_transform(text).toarray())
print('Vocabulary =',vector1.vocabulary_)
print()

vector2 = CountVectorizer(stop_words="english")                           # CounterVectorizer에서 기본적으로 제공하는 Stop Words를 사용하는 방법
print('BoW vector =',vector2.fit_transform(text).toarray())
print('Vocabulary =',vector2.vocabulary_)
print()

stop_words = stopwords.words("english")
vector3 = CountVectorizer(stop_words=stop_words)                           # NLTK에서 기본적으로 제공하는 Stop Words를 사용하는 방법
print('BoW vector =',vector3.fit_transform(text).toarray())
print('Vocabulary =',vector3.vocabulary_)

BoW vector = [[1 1 1 1 1]]
Vocabulary = {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}

BoW vector = [[1 1 1]]
Vocabulary = {'family': 0, 'important': 1, 'thing': 2}

BoW vector = [[1 1 1 1]]
Vocabulary = {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# **TF-IDF, Term Frequency-Inverse Document Frequency**

## **일반 Python 문법을 사용한 TF-IDF 구현**

In [ ]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
]

vocab = list(set(w for doc in docs for w in doc.split()))     # Python의 2중 List Comprehension
vocab.sort()
print(vocab)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [ ]:
N = len(docs)         # 총 문서의 수 

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc

  return log(N/(df+1))

def tf_idf(t, d):
  return tf(t,d) * idf(t)

In [ ]:
## TF 구하기

result = []
for a in range(N):
  result.append([])
  d = docs[a]

  for b in range(len(vocab)):
    t = vocab[b]
    result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns=vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [ ]:
## IDF 구하기

result = []
for i in range(len(vocab)):
  t = vocab[i]
  result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [ ]:
## TF-IDF 구하기

result = []
for a in range(N):
  result.append([])
  d = docs[a]

  for b in range(len(vocab)):
    t = vocab[b]
    result[-1].append(tf_idf(t, d))

tf_idf_ = pd.DataFrame(result, columns=vocab)
tf_idf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


## **Scikit-learn Package를 활용한 DTM, TF-IDF 구현**

In [ ]:
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray())     # 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.vocabulary_)                         # 각 단어와 맵핑된 인덱스 출력
print()

tf_idf_vect = TfidfVectorizer().fit(corpus)       # Scikit-learn에서 제공하는 자동으로 TF-IDF 계산하는 Class
print(tf_idf_vect.transform(corpus).toarray())
print(tf_idf_vect.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
